In this notebook we will use the nn class **Sequential** to model a CNN

We will use the single target network

# Import

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

In [2]:
import load_script

In [3]:
%load_ext autoreload
%autoreload 2

# Loading the dataset

In [4]:
train_dataset,target=load_script.load_dataset(train=True,single_target=True)

In [5]:
test_dataset,test_target=load_script.load_dataset(train=False,single_target=True)

# Added modules

In [33]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

# Defining the network

### Single output

In [6]:
class SingleOutput(nn.Module):
    def __init__(self):
        super(SingleOutput,self).__init__()

        self.conv1=nn.Conv1d(28,14,5)
        self.full1=nn.Linear(14*46,100)
        self.full2=nn.Linear(100,1)
        
    def forward(self,x):
        x=F.max_pool1d(F.relu(self.conv1(x)),1)
        x=x.view(-1)
        x=F.relu(self.full1(x.view(-1,14*46)))
        x=F.sigmoid(self.full2(x))
        
        return x

# Sequential

In [47]:
class Sequential(nn.Module):
    def __init__(self):
        super(Sequential,self).__init__()
        
        layers=[]
        layers.append(nn.Conv1d(28,14,5))
        layers.append(nn.ReLU())
        layers.append(nn.MaxPool1d(1))
        layers.append(Flatten())
        layers.append(nn.Linear(14*46,100))
        layers.append(nn.ReLU())
        layers.append(nn.Linear(100,1))
        layers.append(nn.Sigmoid())
        
        self.sequential=nn.Sequential(*layers)
        
    def forward(self,x):
        x=self.sequential(x)
        
        return x

# Training

In [49]:
net=Sequential()

optimizer = optim.SGD(net.parameters(), lr=0.0001)
criterion=nn.MSELoss()
mini_batch_size=1

for epoch in range(1001):
    
    total_loss=0
    output_target=torch.zeros(target.shape[0])
    for b in range(0,train_dataset.shape[0],mini_batch_size):
        
        
        train_element=train_dataset.narrow(0,b,mini_batch_size)
#         train_element=train_element.view(1,28,-1)
        
        target_element=target.narrow(0,b,mini_batch_size)
        
        optimizer.zero_grad()
        
#         print(train_element.shape)

        out=net(train_element)
        output_target[b:b+mini_batch_size]=(out>0.5).data
#         print(out)
#         print(target_element)
        loss=criterion(out,target_element)
        loss.backward()
        optimizer.step()
        total_loss+=loss.data[0]
        
    output_test=torch.zeros(test_target.shape[0])
    out=net(test_dataset)
#         print(out)
    output_test=(out>0.5).data.type(torch.FloatTensor)
        
    
#     print(type(output_test))
        
    error_train=np.sum(list(output_target==target.data))/target.shape[0]
    error_test=np.sum(list(output_test[:,0]==test_target.data))/test_target.shape[0]
    if epoch%10==0:
        print('Epoch:',epoch,'Loss:',total_loss,'Correct:',str(error_train*100)[:5]+"%",
             'Correct test:',str(error_test*100)[:5]+"%")

Epoch: 0 Loss: 104.91785176397434 Correct: 50.31% Correct test: 49.0%
Epoch: 10 Loss: 62.78841317223214 Correct: 71.51% Correct test: 50.0%
Epoch: 20 Loss: 47.444731049294205 Correct: 83.22% Correct test: 57.0%
Epoch: 30 Loss: 36.23769949478174 Correct: 90.18% Correct test: 59.0%
Epoch: 40 Loss: 27.211828998774763 Correct: 93.98% Correct test: 62.0%
Epoch: 50 Loss: 20.21680698651562 Correct: 95.25% Correct test: 62.0%
Epoch: 60 Loss: 14.899908392814666 Correct: 97.15% Correct test: 63.0%


KeyboardInterrupt: 